In [1]:
from __future__ import annotations

%load_ext autoreload
%autoreload 2

In [2]:
from pathlib import Path

import numpy as np
import pandas as pd

from wufam.data.prepare_data import read_kf_data
from wufam.config.trading_config import TradingConfig
from wufam.dataset import Dataset
from wufam.strategies.base_strategy import BaseStrategy
from wufam.backtest.rolling_backtest import run_rolling_backtest
from wufam.metrics.metrics import calc_sharpe

PATH = Path("../data/kf_data")
START = "1970-01-01"
END = "2024-12-31"
DATASET = Dataset.BM_6_D
FACTORS_DATASET = Dataset.FACTORS_D
WEIGHTING = "value_weighted"
FACTOR_ANNUALIZE = 252

## 1.1

In [3]:
portfolios_total_r, portfolios_xs_r, factors_df, rf = read_kf_data(
    portfolios_filename=PATH / DATASET,
    factors_filename=PATH / FACTORS_DATASET,
    start_date=START,
    end_date=END,
    weighting=WEIGHTING,
)

In [4]:
assert (
    portfolios_total_r.shape[0]
    == portfolios_xs_r.shape[0]
    == factors_df.shape[0]
    == rf.shape[0]
)

## 1.2

In [5]:
from wufam.strategies.heuristics.equally_weighted import EWStrategy
from wufam.strategies.optimized.mean_var import MeanVariance
from wufam.strategies.optimized.min_var import MinVariance

trading_config = TradingConfig(total_exposure=1)

ew_strategy = EWStrategy()

In [6]:
from wufam.estimation.mean.sample_mu_estimator import SampleMuEstimator
from wufam.estimation.covariance.sample_cov_estimator import SampleCovEstimator

mv_strategy = MeanVariance(
    mu_estimator=SampleMuEstimator(),
    cov_estimator=SampleCovEstimator(),
    trading_config=trading_config,
    window_size=365,
)

In [7]:
min_var_strategy = MinVariance(
    cov_estimator=SampleCovEstimator(),
    trading_config=trading_config,
    window_size=365,
)

In [8]:
trading_config_min_var = TradingConfig(total_exposure=1, min_exposure=0.0)

min_var_c_strategy = MinVariance(
    cov_estimator=SampleCovEstimator(),
    trading_config=trading_config_min_var,
    window_size=365,
)

In [9]:
from wufam.estimation.covariance.shrinkage.lw_cv_cov_estimator import (
    LedoitWolfCVCovEstimator,
)

min_var_lw_strategy = MinVariance(
    cov_estimator=LedoitWolfCVCovEstimator(),
    trading_config=trading_config,
    window_size=365,
)

In [10]:
from wufam.estimation.covariance.shrinkage.qis import QISCovEstimator

min_var_qis_strategy = MinVariance(
    cov_estimator=QISCovEstimator(),
    trading_config=trading_config,
    window_size=365,
)

In [11]:
min_var_lw_cv_strategy = MinVariance(
    cov_estimator=LedoitWolfCVCovEstimator(alphas=np.linspace(0.0, 1.0, 100)),
    trading_config=trading_config,
    window_size=365,
)

In [12]:
from wufam.estimation.covariance.factor.factor_cov_estimator import FactorCovEstimator
from wufam.estimation.covariance.shrinkage.pca_cov_estimator import PCACovEstimator

factor_cov_estimator = FactorCovEstimator(
    factor_cov_estimator=SampleCovEstimator(),
    residual_cov_estimator=PCACovEstimator(),
    factors_selection=["Mkt-RF"],
)

min_var_factor_strategy = MinVariance(
    cov_estimator=factor_cov_estimator,
    trading_config=trading_config,
    window_size=365,
)

In [13]:
from wufam.strategies.timed.vol_managed_strategy import VolManagedStrategy

vol_managed_trading_config = TradingConfig(min_exposure=0.0, max_exposure=2.0)

vol_managed_strategy = VolManagedStrategy(
    trading_config=vol_managed_trading_config,
    vol_window=21,
)

In [27]:
def evaluate_strategy(
    strategy: BaseStrategy, return_weights: bool = False
) -> tuple[float, float] | tuple[float, float, pd.DataFrame]:
    bt_res = run_rolling_backtest(
        strategy=strategy,
        excess_returns=portfolios_xs_r,
        factors=factors_df,
        rf=rf,
        freq="M",
        trading_lag=1,
        return_weights=return_weights,
    )

    if return_weights:
        total_r, turnover, me_weights = bt_res
    else:
        total_r, turnover = bt_res

    sr = calc_sharpe(
        strategy_total_r=total_r,
        rf_rate=rf,
        factor_annualize=FACTOR_ANNUALIZE,
    )

    if return_weights:
        return sr, turnover.mean(), me_weights
    return sr, turnover.mean()

In [15]:
evaluate_strategy(ew_strategy)

Optimizing Strategy: 100%|██████████| 659/659 [00:00<00:00, 1972.69it/s]


(0.4402388970354759, np.float64(0.015229700593042371))

In [29]:
sr, turnover, w = evaluate_strategy(mv_strategy, return_weights=True)
sr, turnover

Optimizing Strategy: 100%|██████████| 659/659 [00:00<00:00, 702.11it/s]


(nan, np.float64(67.45486112038247))

In [30]:
w.isna().any().any()

np.False_

In [31]:
w.abs().max()

SMALL LoBM    33395.285223
ME1 BM2       49685.128306
SMALL HiBM    13233.415134
BIG LoBM      10825.078693
ME2 BM2       18102.148982
BIG HiBM      62568.928105
dtype: object

In [17]:
evaluate_strategy(min_var_strategy)

Optimizing Strategy: 100%|██████████| 659/659 [00:00<00:00, 931.05it/s]


(1.0333752498077793, np.float64(0.5021140026630577))

In [18]:
evaluate_strategy(min_var_c_strategy)

Optimizing Strategy: 100%|██████████| 659/659 [00:04<00:00, 144.02it/s]


(0.6504575380198405, np.float64(0.11393304146659755))

In [19]:
evaluate_strategy(min_var_lw_strategy)

Optimizing Strategy: 100%|██████████| 659/659 [00:00<00:00, 713.69it/s]


(0.9573740069231945, np.float64(0.32174626994432326))

In [20]:
evaluate_strategy(min_var_factor_strategy)

Optimizing Strategy: 100%|██████████| 659/659 [00:04<00:00, 141.26it/s]


(1.067201667960036, np.float64(0.6201519831392531))

In [21]:
evaluate_strategy(min_var_qis_strategy)

Optimizing Strategy: 100%|██████████| 659/659 [00:01<00:00, 425.65it/s]


(0.9973516764669229, np.float64(0.619142265968935))

In [22]:
evaluate_strategy(vol_managed_strategy)

Optimizing Strategy: 100%|██████████| 659/659 [00:00<00:00, 904.97it/s] 


(0.5662518243684176, np.float64(0.4349921576764195))

In [23]:
evaluate_strategy(min_var_lw_cv_strategy)

Optimizing Strategy: 100%|██████████| 659/659 [02:54<00:00,  3.78it/s]


(0.9831891235641731, np.float64(0.6440725344688129))

In [24]:
min_var_lw_cv_strategy.cov_estimator.best_alpha

np.float64(0.010101010101010102)

In [25]:
trading_config_naive_min_var = TradingConfig(total_exposure=1, min_exposure=0.5 / 25)

min_var_naive_c_strategy = MinVariance(
    cov_estimator=SampleCovEstimator(),
    trading_config=trading_config_naive_min_var,
    window_size=None,
)

In [26]:
evaluate_strategy(min_var_naive_c_strategy)

Optimizing Strategy: 100%|██████████| 659/659 [00:05<00:00, 119.74it/s]


(0.5226548451417654, np.float64(0.024205558918543195))